# <CENTER> СЛУЧАЙНЫЙ ЛЕС

## <CENTER> Проект "Предсказание прогноза погоды на завтра"

Пожалуй, каждый из нас практически ежедневно смотрит прогноз погоды: будет сегодня тепло или холодно, брать ли зонт? Мы расстраиваемся, если внезапно идёт дождь, а мы оказались к этому не готовы. Иногда можно понять, что будет дождь, просто взглянув на небо, но часто такие предположения оказываются неверными. Надёжнее всего пользоваться прогнозами, которые публикуют специалисты. А задумывались ли вы, как формируются эти прогнозы?

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков шесть — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой. Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра. Целевой переменной является столбец RainTomorrow. Значение этой переменной мы и будем пытаться предсказать.

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import sys
import re
import statistics
import json
from itertools import combinations
from pylab import rcParams

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import randint
from scipy.stats import bernoulli
from scipy.stats import norm, uniform

from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn import cluster
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
data = pd.read_csv('data/weatherAUS.csv', sep=',')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Познакомимся с данными:

- Date — дата, в которую зафиксировано наблюдение;
- Location — местонахождение метеорологической станции;
- MinTemp — минимальная температура (℃);
- MaxTemp — максимальная температура (℃);
- Rainfall — количество осадков (дождь) за сутки (мм);
- Evaporation — количество испарений до 9 утра (мм);
- Sunshine — количество часов в сутках, когда светило солнце;
- WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
- WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
- WindDir9am — направление ветра в 9 утра;
- WindDir3pm — направление ветра в 3 часа дня;
- WindSpeed9am — скорость ветра в 9 часов утра;
- WindSpeed3pm — скорость ветра в 3 часа дня;
- Humidity9am — влажность в 9 утра;
- Humidity3pm — влажность в 3 часа дня;
- Pressure9am — атмосферное давление в 9 утра;
- Pressure3pm — атмосферное давление в 3 часа дня;
- Cloud9am — часть неба, закрытая облаками, в 9 утра;
- Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
- Temp9am — температура в 9 утра;
- Temp3pm — температура в 3 часа дня;
- RainToday — наличие дождя в этот день;
- RainTomorrow — наличие дождя на следующий день.

Посмотрим сколько суммарно пропусков в данных:

In [4]:
data.isnull().sum().sum()

343248

В некоторых признаках пропусков более 40 % — удалите такие признаки.

In [5]:
round(data.isna().sum() / len(data), 3)
data.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)

Теперь обработаем признаки `RainToday` и `RainTomorrow` таким образом, чтобы вместо yes было значение 1, а вместо no — значение 0. Вычислим среднее арифметическое для преобразованного признака RainToday и запишим его в ответ

In [6]:
data.RainToday = data.RainToday.map({'No': 0, 'Yes': 1})
data.RainTomorrow = data.RainTomorrow.map({'No': 0, 'Yes': 1})

data['RainToday'].mean()

0.22419285648984874

Обработаем признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалим. Определим, в какой месяц выпадает больше всего дождей.

In [7]:
data.Date = pd.to_datetime(data.Date)
data['Month'] = data.Date.dt.month
data.drop('Date', axis = 1, inplace = True)
data_season = data.groupby('Month').mean()
data_season[['RainToday']]

,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


Обработаем оставшиеся категориальные признаки. С помощью метода `get_dummies` с настройками по умолчанию создадим `dummy-переменные` для всех категориальных признаков (их пять), которые есть в данных на этот момент. Кодировку признаков важно выполнить именно в следующем порядке: categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. Это необходимо для того, чтобы наши дальнейшие ответы сходились с нашим решением, так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов. 

In [8]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data_dummies = pd.get_dummies(data, columns=categoricals)
data_dummies.shape

(145460, 124)

Удалим все строки, где есть пропуски. Далее разобъем данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмем число 31. Посмотрим каково среднее значение целевой переменной на тестовой выборке

In [9]:
data_dummies.dropna(inplace=True)
X = data_dummies.drop('RainTomorrow', axis = 1)
Y = data_dummies['RainTomorrow']  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 31)
Y_test.mean()

0.22770253002811142

### Бутстреп

Теперь давайте вспомним про `бутстреп`. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную». Сделаем оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку `MinTemp`). Для этого сгенерируем 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используем `np.random.randint()`: сгенерируем необходимое количество индексов и по ним извлечем соответствующие элементы выборки. Случайность фиксируем с помощью `np.random.seed(31)`. Для каждой выборки вычислим среднее значение, а после найдем стандартное отклонение для этих значений.

In [10]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

### Логистическая регрессия

Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую `логистическую регрессию` (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами. В качестве ответа введите значение метрики `roc_auc` на тестовой выборке.

In [11]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(Y_test, preds_test)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.726396963482411

### Дерево решений

Теперь попробуем обучить на наших данных другой алгоритм — дерево решений. С помощью GridSearchCV сделаем перебор гиперпараметров по следующей сетке:

In [12]:
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

Для параметра кросс-валидации `cv` зададим значение 3. Для решающего дерева определим параметр `random_state=42`. Остальные параметры оставим по умолчанию.

Вычислим значение `roc_auc` для решающего дерева с гиперпараметрами, определёнными в качестве оптимальных. 

In [13]:
clf = tree.DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 3))

0.703


Найдем оптимальные значения гиперпараметров:

In [14]:
from sklearn.model_selection import GridSearchCV
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

### Случайный лес

К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей. Теперь построим случайный лес, включающий 100 деревьев. Зададим параметр random_state=31. Остальные параметры оставим по умолчанию. Посмотрим какая теперь будет метрика roc_auc на тестовой выборке?

In [15]:

clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

0.73


Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие:`'max_features'`, `'min_samples_leaf'`, `'max_depth'`. Возьмем случайный лес из 100 деревьев и найдем оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая: {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]} Перебор осуществим с помощью `GridSearchCV`. Для параметра кросс-валидации cv зададим значение 3. Случайности фиксируем параметром `random_state = 31`. Остальные значения оставим по умолчанию. Посмотрим какое значение roc_auc получилось для оптимальных гиперпараметров?

In [16]:
params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)
clf = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   0.8s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.8s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.8s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   0.8s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.9s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.8s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   0.8s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.8s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.8s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   0.8

Благодаря случайному лесу можно узнать, какие признаки оказывают большее влияние на целевую переменную по сравнению с другими. Оценим значимость признаков

In [17]:
feature_names = [x for x in data_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
